In [1]:
import os
import scanpy as sc
import pandas as pd
import scglue
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
import anndata as ad

/home/xuzekai/mambaforge/envs/ATAC_env1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/xuzekai/mambaforge/envs/ATAC_env1/lib/python3.9/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [29]:
DAR=pd.read_csv('/media/AnalysisDisk1/xuzekai/20250715_BMI/final_result/DAR_final_sig.csv')

In [11]:
len(DAR[DAR['comparison_ATAC']=='Underweight_vs_Normal']['peak'].unique())

8440

In [16]:
count = DAR[DAR['comparison_ATAC'] == 'Underweight_vs_Normal']['peak'].str.startswith('chrY')

count

0         True
1         True
2         True
5         True
6         True
         ...  
33323    False
33324    False
33325    False
33326    False
33327    False
Name: peak, Length: 11338, dtype: bool

In [24]:
result = DAR[
    (DAR['comparison_ATAC'] == 'Overweight_vs_Normal') & 
    (DAR['peak'].str.startswith('chrY'))
]
len(result['peak'].unique())

28

In [25]:
result = DAR[
    (DAR['comparison_ATAC'] == 'Underweight_vs_Normal') & 
    (DAR['peak'].str.startswith('chrY'))
]
len(result['peak'].unique())

76

In [30]:
split = DAR['peak'].str.split(r"[:-]")
DAR["chrom"] = split.map(lambda x: x[0])
DAR["chromStart"] = split.map(lambda x: x[1]).astype(int)
DAR["chromEnd"] = split.map(lambda x: x[2]).astype(int)

In [ ]:
DAR=DAR[['chrom','chromStart','chromEnd']]

In [31]:
DAR=DAR[['chrom','chromStart','chromEnd']]
unique_peaks = DAR.drop_duplicates(subset=['chrom', 'chromStart', 'chromEnd'], keep='first')
unique_peaks.to_csv(
            '/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_enrichment/DAR_unique_peaks.bed',
            sep='\t',
            header=False,
            index=False,
            columns=['chrom', 'chromStart', 'chromEnd'])

,Unnamed: 0,p_val_ATAC,log2FC_ATAC,p_val_adj_ATAC,comparison_ATAC,peak,celltype_ATAC,comp,control,chrom,chromStart,chromEnd
0,0,1.799742e-15,-4.348075,6.083777e-10,Underweight_vs_Normal,chrY:12904575-12905076,CD4_Treg_FCRL3,Underweight,Normal,chrY,12904575,12905076
1,1,4.509594e-11,-2.634610,1.524405e-05,Underweight_vs_Normal,chrY:20575413-20575914,CD4_Treg_FCRL3,Underweight,Normal,chrY,20575413,20575914
2,2,4.035983e-09,-2.395256,1.364308e-03,Underweight_vs_Normal,chrY:7273539-7274040,CD4_Treg_FCRL3,Underweight,Normal,chrY,7273539,7274040
3,3,4.173088e-09,-3.359624,1.410654e-03,Underweight_vs_Overweight,chrY:20575413-20575914,CD4_Treg_FCRL3,Underweight,Overweight,chrY,20575413,20575914
4,4,9.946644e-08,-4.406483,3.362324e-02,Underweight_vs_Overweight,chrY:12904575-12905076,CD4_Treg_FCRL3,Underweight,Overweight,chrY,12904575,12905076
...,...,...,...,...,...,...,...,...,...,...,...,...
33524,43533,2.092217e-07,10.388479,7.072446e-02,Underweight_vs_Overweight,chr16:81578042-81578543,Unswitched_Bm_CD1C,Underweight,Overweight,chr16,81578042,81578543
33525,43534,2.447439e-07,4.547295,8.273224e-02,Underweight_vs_Overweight,chr7:44631362-44631863,Unswitched_Bm_CD1C,Underweight,Overweight,chr7,44631362,44631863
33526,43535,2.463935e-07,-2.429886,8.328988e-02,Underweight_vs_Overweight,chr5:74839170-74839671,Unswitched_Bm_CD1C,Underweight,Overweight,chr5,74839170,74839671
33527,43536,2.574729e-07,3.842864,8.703511e-02,Underweight_vs_Overweight,chr20:45405444-45405945,Unswitched_Bm_CD1C,Underweight,Overweight,chr20,45405444,45405945


In [33]:
celltypes = DAR['celltype_ATAC'].unique()

for celltype in celltypes:
    # 筛选出当前细胞类型的数据
    DAR_celltype = DAR[DAR['celltype_ATAC'] == celltype][['chrom', 'chromStart', 'chromEnd']]
    
    # 删除重复的peaks
    unique_peaks = DAR_celltype.drop_duplicates(subset=['chrom', 'chromStart', 'chromEnd'], keep='first')
    
    # 构建输出文件路径
    output_path = f'/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/{celltype}.bed'
    
    # 保存为 .bed 文件
    unique_peaks.to_csv(
        output_path,
        sep='\t',
        header=False,
        index=False,
        columns=['chrom', 'chromStart', 'chromEnd']
    )
    print(f"Saved {output_path}")

Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_Treg_FCRL3.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/Atypical_Bm_ITGAX.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/Bn_TCL1A.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_CTL_GZMH.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_Tcm_CXCR5.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_Tcm_IFI44L.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_Tem_CCR7neg.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_Tfh-like_CXCR5.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_Th1-like_GZMK.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_Th17-like_RORC.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_Th_LMNA.bed
Saved /media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/CD4_Tn_CCR7.bed
Saved /media/AnalysisDisk1/xuzekai/20250